In [1]:
import os, re, pulp, json, time
os.environ["GRB_LICENSE_FILE"] = "/Users/gabrielaidar/Desktop/Mestrado/outros/documentos/gurobi.lic"

import pandas as pd
from collections import defaultdict
from datetime import date


In [ ]:
DADOS_PATH = "../dados/"
COBERTURA = 0.9  # Quanto das escolas será alocado


hoje = date.today()
timestamp = str(time.time())[-7:]
data = f"{hoje.year}_{hoje.month}_{hoje.day}_{timestamp}"

nome_arquivo_resultado = f"{DADOS_PATH}resultados/{data}_resultado.json"
nome_arquivo_log = f"{DADOS_PATH}resultados/{data}_log.txt"


In [3]:
df = pd.read_csv(f"{DADOS_PATH}temporarios/03-pronto_po.csv",
                         index_col="CO_ENTIDADE")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37178 entries, 11006765 to 53086007
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   motivacao           37178 non-null  int64
 1   Arthur Lincoln      37178 non-null  int64
 2   Ana Claudia         37178 non-null  int64
 3   Elina Maciel        37178 non-null  int64
 4   Galdino Espindola   37178 non-null  int64
 5   Niedna Leite        37178 non-null  int64
 6   Ricardo Ricato      37178 non-null  int64
 7   Thiago Souza        37178 non-null  int64
 8   Eulanda Costa       37178 non-null  int64
 9   Lis Lucio           37178 non-null  int64
 10  Rafaelle Magalhães  37178 non-null  int64
 11  Flavio Paim         37178 non-null  int64
 12  Adriana Mota        37178 non-null  int64
 13  Reginaldo Machado   37178 non-null  int64
 14  Tizziana Dias       37178 non-null  int64
 15  Marcelo Trinta      37178 non-null  int64
 16  Fabricio Salomao    37178 non-null 

In [4]:

# --- MODELO ---
modelo = pulp.LpProblem("Poliedro", pulp.LpMinimize)

# --- DADOS ---
distancias = df.drop(columns="motivacao")
motivacao = df["motivacao"]
consultores = distancias.columns.to_list()
escolas = df.index.tolist()
mm = sum(motivacao)/len(consultores)


# --- VARIÁVEL ---
x = pulp.LpVariable.dicts("x", [(i, j) for i in escolas for j in consultores],
                          lowBound=0, cat="Binary")


# --- FUNÇÃO OBJETIVO ---
modelo += pulp.lpSum(distancias.loc[i, j] * x[(i, j)]
                    for i in escolas
                    for j in consultores)


# --- RESTRIÇÕES ---
for i in escolas:  # Cada escola é atribuida a apenas um consultor
    modelo += pulp.lpSum(x[(i, j)] for j in consultores) == 1
for j in consultores:
    modelo += pulp.lpSum(x[(i, j)]*motivacao[i] for i in escolas) >= COBERTURA * mm


# --- SOLUÇÃO ---
modelo.solve(pulp.GUROBI(
                    logPath=nome_arquivo_log,
                    # gapRel=0.01,
                    timeLimit=60,
                    ))


# --- RESULTADOS ---
print("\n--------- RESULTADOS ---------")
print(f"Valor FO = {modelo.objective.value():.0f}")


Set parameter Username
Set parameter LicenseID to value 2635692
Academic license - for non-commercial use only - expires 2026-03-12
Set parameter TimeLimit to value 60
Set parameter LogFile to value "../dados/resultados/2025_11_26_060524_log.txt"
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 25.1.0 25B78)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  60

Optimize a model with 37202 rows, 892272 columns and 1784544 nonzeros
Model fingerprint: 0x86605e23
Variable types: 0 continuous, 892272 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [1e+00, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+06]
Presolve time: 2.04s
Presolved: 37202 rows, 892272 columns, 1784544 nonzeros
Variable types: 0 continuous, 892272 integer (892272 binary)
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barri

TypeError: unsupported format string passed to NoneType.__format__


100 linhas:
- Gap 1% =  207.502, tempo = 1s  
- Otima = 206.564, tempo = 100s  

1.000 linhas:
- Gap 1% =  1.483.398, tempo = 1s  
- Gap 0.74% Interrompi = 1.470.461, tempo = 12min

37.178 linhas (tudo):
- Gap 1% =  6.765.066, tempo = 30s
- "Otima" = 6.760.710, tempo = 36s  



In [ ]:
alocacoes = defaultdict(list)
padrao = re.compile(r"x_\((\d+),_'([^']+)'\)")

for v in modelo.variables():
    val = v.value()
    if val != 0:
        m = padrao.match(v.name)
        nome = m.group(2)
        escola = m.group(1)
        
        alocacoes[nome].append(escola)


# for nome, codigos in alocacoes.items():
#     print(f"{nome}: {codigos}")


## Salvando o resultado

In [ ]:
with open(nome_arquivo_resultado, "w", encoding="utf-8") as f:
    json.dump(alocacoes, f, ensure_ascii=False, indent=4)